In [7]:
# Лабораторная работа 1 по дисциплине МРЗвИС
# Выполнена студентом группы 121702
# БГУИР Заломов Роман Андреевич
#
# Вариант 15: Реализовать модель линейной рециркуляционной сети 
# с постоянным коэффициентом обучения и нормированными весовыми коэффициентами.
#
# Данный файл содержит алгоритмы построения графиков различных зависимостей

In [8]:
from lrnn import (
    LRNN,
    image_to_blocks,
    compress_image,
    decompress_image
)
import numpy as np
from PIL import Image
import plotly.graph_objects as go

In [9]:
MAX_RGB_VALUE = 255
COLOR_CHANNELS_AMOUNT = 3

In [10]:
# Collecting everything

block_width = 10
block_height = 10

n = block_height * block_width
# Hidden layer neuron amount
p = 20 

img = Image.open('car.jpg')
img_array = np.asarray(img)
shape = img_array.shape
blocks = image_to_blocks(img_array, block_height, block_width, overlap=0)

l = len(blocks)
# Compression coeff
print('Z =', (n*l) / ((n+l) * p+2))

color_df = ((2 * blocks / MAX_RGB_VALUE) - 1).reshape(len(blocks), -1, 3).transpose(0, 2, 1).reshape(-1, block_height * block_width)
train = np.matrix(color_df[np.random.choice(color_df.shape[0], int(color_df.shape[0] * 0.1))])


network = LRNN(n, p, 0.001)
network.train(train, 15000, learn_by_loss=True, max_loss=2000)

compressed = compress_image(network.W_enc, img_array, COLOR_CHANNELS_AMOUNT, block_height, block_width, 0)

compression_info_size = (
    compressed.size * compressed.itemsize +
    network.W_dec.size * network.W_dec.itemsize +
    np.array(shape).size * np.array(shape).itemsize +
    np.array((block_height, block_width)).size * np.array((block_height, block_width)).itemsize 
) * 8
print(f'Compression coefficient: {(img_array.size * img_array.itemsize * 8) / compression_info_size}')
print('Z =', (n*l) / ((n+l) * p + 2))

dimg = decompress_image(network.W_dec, compressed, shape, COLOR_CHANNELS_AMOUNT, block_height, block_width, 0)
dimg_array = np.asarray(dimg)
dimg.save('compression-decompression_test.jpg')

Z = 4.807507403561401
Epoch 1/15000, Loss: 2643.660627917333
Epoch 2/15000, Loss: 2388.4832074880987
Epoch 3/15000, Loss: 2262.88011232667
Epoch 4/15000, Loss: 2180.2016166577155
Epoch 5/15000, Loss: 2113.9801737655744
Epoch 6/15000, Loss: 2062.730749948625
Epoch 7/15000, Loss: 2016.5568945080618
Epoch 8/15000, Loss: 1974.6083890575921
Compression coefficient: 2.466780686751743
Z = 4.807507403561401


## Relation between learning rate and epochs 

In [11]:
alphas = (0.0005, 0.0006, 0.0007, 0.0008, 0.0009, 0.001)
epochs_amount: list[int] = []
for alpha in alphas:
    dl = LRNN(n, p, alpha)
    dl.train(train, 15000, learn_by_loss=True, max_loss=2500)
    epochs_amount.append(dl.epoch)
fig = go.Figure()
fig.add_trace(go.Scatter(x=alphas, y=epochs_amount, mode='lines+markers'))
fig.update_layout(title='Зависимость количества итераций обучения от коэффициента обучения',
                  xaxis_title='Коэффициент обучения',
                  yaxis_title='Количество итераций обучения',
                  height=720, width=1280,
                  font=dict(size=20))
fig.show()

Epoch 1/15000, Loss: 4791.536501632571
Epoch 2/15000, Loss: 3801.206481195449
Epoch 3/15000, Loss: 3402.8885583780575
Epoch 4/15000, Loss: 3169.307599432671
Epoch 5/15000, Loss: 3010.818751850252
Epoch 6/15000, Loss: 2892.621724158032
Epoch 7/15000, Loss: 2800.334100442636
Epoch 8/15000, Loss: 2726.2994060637225
Epoch 9/15000, Loss: 2662.788763506726
Epoch 10/15000, Loss: 2606.887768870418
Epoch 11/15000, Loss: 2558.729268982725
Epoch 12/15000, Loss: 2515.6489744712285
Epoch 13/15000, Loss: 2476.1407290913353
Epoch 1/15000, Loss: 4316.146037361944
Epoch 2/15000, Loss: 3517.8978476408397
Epoch 3/15000, Loss: 3146.632914621037
Epoch 4/15000, Loss: 2926.7492494495746
Epoch 5/15000, Loss: 2774.3286260204463
Epoch 6/15000, Loss: 2660.3352712520145
Epoch 7/15000, Loss: 2570.4073667816747
Epoch 8/15000, Loss: 2495.1746661810653
Epoch 1/15000, Loss: 2892.2990422741345
Epoch 2/15000, Loss: 2686.809015198793
Epoch 3/15000, Loss: 2578.568081886334
Epoch 4/15000, Loss: 2501.9662556840503
Epoch 5/1

## Relationship between hidden layer dimension and epochs

In [12]:
p_s = (10, 15, 20, 25, 30, 35, 40)
epochs_amount: list[int] = []
compression_rates: list[float] = []
for p in p_s:
    compression_rates.append((n*l) / ((n+l) * p + 2))
    dl = LRNN(n, p, 0.001)
    dl.train(train, 15000, learn_by_loss=True, max_loss=2000)
    epochs_amount.append(dl.epoch)
fig = go.Figure()
fig.add_trace(go.Scatter(x=compression_rates, y=epochs_amount, mode='lines+markers'))
fig.update_layout(title='Зависимость количества итераций обучения от коэффициента сжатия',
                  xaxis_title='Коэффициент сжатия',
                  yaxis_title='Количество итераций обучения',
                  height=720, width=1280,
                  font=dict(size=20))
fig.show()

Epoch 1/15000, Loss: 3389.9609106053326
Epoch 2/15000, Loss: 2795.76864363157
Epoch 3/15000, Loss: 2547.859915828938
Epoch 4/15000, Loss: 2421.031577586499
Epoch 5/15000, Loss: 2342.7152640161958
Epoch 6/15000, Loss: 2287.875317685338
Epoch 7/15000, Loss: 2248.6729535452
Epoch 8/15000, Loss: 2218.2840994135568
Epoch 9/15000, Loss: 2195.319457131569
Epoch 10/15000, Loss: 2173.1878688083098
Epoch 11/15000, Loss: 2156.3987447139257
Epoch 12/15000, Loss: 2140.901089799217
Epoch 13/15000, Loss: 2124.274210936122
Epoch 14/15000, Loss: 2111.780207152004
Epoch 15/15000, Loss: 2097.797073039105
Epoch 16/15000, Loss: 2085.659058462184
Epoch 17/15000, Loss: 2074.137886517042
Epoch 18/15000, Loss: 2062.0938519072233
Epoch 19/15000, Loss: 2051.3129557167226
Epoch 20/15000, Loss: 2040.5475012529719
Epoch 21/15000, Loss: 2031.129864144571
Epoch 22/15000, Loss: 2021.3206155594576
Epoch 23/15000, Loss: 2011.0679452108861
Epoch 24/15000, Loss: 2000.8957510652083
Epoch 25/15000, Loss: 1990.6423877542234


## Relationship between max error and epochs

In [13]:
max_errors = list(range(1000, 3000, 100))
epochs_amount: list[int] = []
for err in max_errors:    
    dl = LRNN(n, 50, 0.001)
    dl.train(train, 15000, learn_by_loss=True, max_loss=err)
    epochs_amount.append(dl.epoch)
fig = go.Figure()
fig.add_trace(go.Scatter(x=max_errors, y=epochs_amount, mode='lines+markers'))
fig.update_layout(title='Зависимость количества итераций обучения от значения максимальной ошибки',
                  xaxis_title='Значение максимальной ошибки',
                  yaxis_title='Количество итераций обучения',
                  height=720, width=1280,
                  font=dict(size=20))
fig.show()

Epoch 1/15000, Loss: 2797.233974282863
Epoch 2/15000, Loss: 2317.191328759601
Epoch 3/15000, Loss: 2058.269929736794
Epoch 4/15000, Loss: 1875.6849355857407
Epoch 5/15000, Loss: 1738.6966476119844
Epoch 6/15000, Loss: 1631.6152063939141
Epoch 7/15000, Loss: 1543.4793864646997
Epoch 8/15000, Loss: 1469.6743861146742
Epoch 9/15000, Loss: 1406.14212906825
Epoch 10/15000, Loss: 1350.9568982113444
Epoch 11/15000, Loss: 1303.2832552788152
Epoch 12/15000, Loss: 1259.5769059170432
Epoch 13/15000, Loss: 1220.9126917525537
Epoch 14/15000, Loss: 1186.1421572858615
Epoch 15/15000, Loss: 1155.3205219823396
Epoch 16/15000, Loss: 1128.0344914886166
Epoch 17/15000, Loss: 1103.7840856840073
Epoch 18/15000, Loss: 1081.5774256533437
Epoch 19/15000, Loss: 1060.5574108442515
Epoch 20/15000, Loss: 1041.7562947609474
Epoch 21/15000, Loss: 1025.1828552050142
Epoch 22/15000, Loss: 1009.9789293354818
Epoch 23/15000, Loss: 995.6714809424357
Epoch 1/15000, Loss: 2766.8178792458743
Epoch 2/15000, Loss: 2303.867870

## Relationship between image and epochs

In [14]:
# Collecting everything

block_width = 10
block_height = 10

n = block_height * block_width
# Hidden layer neuron amount
p = 50 

img = Image.open('images/winter.jpg')
img_array = np.asarray(img)
shape = img_array.shape
blocks = image_to_blocks(img_array, block_height, block_width, overlap=0)

l = len(blocks)
# Compression coeff
print('Z =', (n*l) / ((n+l) * p+2))

color_df = ((2 * blocks / MAX_RGB_VALUE) - 1).reshape(len(blocks), -1, 3).transpose(0, 2, 1).reshape(-1, block_height * block_width)
train = np.matrix(color_df[np.random.choice(color_df.shape[0], int(color_df.shape[0] * 0.05))])

network = LRNN(n, p, 0.001)
network.train(train, 15000, learn_by_loss=True, max_loss=1500)

print(network.epoch)

# compressed = compress_image(network.W_enc, img_array, COLOR_CHANNELS_AMOUNT, block_height, block_width, 0)
# dimg = decompress_image(network.W_dec, compressed, shape, COLOR_CHANNELS_AMOUNT, block_height, block_width, 0)
# dimg_array = np.asarray(dimg)
# dimg.save('compression-decompression_test.jpg')

Z = 1.9230473377332657
Epoch 1/15000, Loss: 7249.204110497766
Epoch 2/15000, Loss: 3880.151918643388
Epoch 3/15000, Loss: 3119.007891899047
Epoch 4/15000, Loss: 2764.050247474703
Epoch 5/15000, Loss: 2530.548851292371
Epoch 6/15000, Loss: 2357.6743368277907
Epoch 7/15000, Loss: 2222.6687505956697
Epoch 8/15000, Loss: 2112.1283245546465
Epoch 9/15000, Loss: 2018.8318164960942
Epoch 10/15000, Loss: 1939.6484548062735
Epoch 11/15000, Loss: 1872.055842951379
Epoch 12/15000, Loss: 1812.023755101229
Epoch 13/15000, Loss: 1759.6028877987903
Epoch 14/15000, Loss: 1712.2467112904692
Epoch 15/15000, Loss: 1669.5723799791704
Epoch 16/15000, Loss: 1631.1716563270709
Epoch 17/15000, Loss: 1595.9001508003012
Epoch 18/15000, Loss: 1563.9729390540192
Epoch 19/15000, Loss: 1533.541698407911
Epoch 20/15000, Loss: 1505.618591047944
Epoch 21/15000, Loss: 1479.3376602011897
21
